In [1]:
#%pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]
#%pip install gym[accept-rom-license]
#%pip install pyglet

In [1]:
import gym
import random

In [2]:
env = gym.make('MsPacman-v0')
#env = gym.make('ALE/MsPacman-v5')#, render_mode='human')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

# sprawdzić evny

In [3]:
env.unwrapped.get_action_meanings()

['NOOP',
 'UP',
 'RIGHT',
 'LEFT',
 'DOWN',
 'UPRIGHT',
 'UPLEFT',
 'DOWNRIGHT',
 'DOWNLEFT']

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice(range(actions))
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

C:\Users\milod\anaconda3\envs\tf_gpu\lib\site-packages\gym\envs\atari\environment.py:255: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(
C:\Users\milod\anaconda3\envs\tf_gpu\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Po ustawieniu trybu wątku nie można go zmienić
  warnings.warn(str(err))


Episode:1 Score:250.0
Episode:2 Score:210.0
Episode:3 Score:240.0
Episode:4 Score:170.0
Episode:5 Score:340.0


In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam, SGD

In [7]:
def build_model(height, width, channels, actions):
    model = Sequential(layers=[
        Convolution2D(32, (8, 8), strides = (4,4), activation='relu', input_shape = (3,height,width,channels)),
        Convolution2D(64, (4, 4), strides = (2,2), activation='relu'),
        Convolution2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(actions, activation='linear')])
    print(model.summary())
    return model


In [8]:
(height, width)

(210, 160)

In [9]:
model = build_model(height, width, channels, actions)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten (Flatten)            (None, 67584)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               34603520  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 2

In [10]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [11]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=10000)
    memory = SequentialMemory(limit=100000, window_length=3) #limit = 100 000
    dqn = DQNAgent(
        model=model, 
        memory=memory, 
        policy=policy, 
        enable_dueling_network=True, 
        dueling_type='avg', 
        nb_actions=actions,
        nb_steps_warmup=1000
    )
    return dqn

In [12]:
del model
model = build_model(height, width, channels, actions)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_5 (Dense)              (None, 9)                

In [13]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-4))

In [14]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [15]:
tf.debugging.set_log_device_placement(True)

In [16]:
with tf.device('/GPU:0'):
    dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)

Training for 50000 steps ...


C:\Users\milod\anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


   624/50000: episode: 1, duration: 7.534s, episode steps: 624, steps per second:  83, episode reward: 210.000, mean reward:  0.337 [ 0.000, 10.000], mean action: 3.870 [0.000, 8.000],  loss: --, mean_q: --, mean_eps: --


C:\Users\milod\anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  1274/50000: episode: 2, duration: 145.707s, episode steps: 650, steps per second:   4, episode reward: 190.000, mean reward:  0.292 [ 0.000, 10.000], mean action: 4.080 [0.000, 8.000],  loss: 54.115806, mean_q: 16.609144, mean_eps: 0.897670
  2336/50000: episode: 3, duration: 548.229s, episode steps: 1062, steps per second:   2, episode reward: 600.000, mean reward:  0.565 [ 0.000, 200.000], mean action: 4.217 [0.000, 8.000],  loss: 4.375737, mean_q: 15.213529, mean_eps: 0.837595
  2882/50000: episode: 4, duration: 284.823s, episode steps: 546, steps per second:   2, episode reward: 150.000, mean reward:  0.275 [ 0.000, 10.000], mean action: 3.813 [0.000, 8.000],  loss: 8.238887, mean_q: 15.116696, mean_eps: 0.765235
  3612/50000: episode: 5, duration: 377.314s, episode steps: 730, steps per second:   2, episode reward: 400.000, mean reward:  0.548 [ 0.000, 10.000], mean action: 4.273 [0.000, 8.000],  loss: 3.364704, mean_q: 15.662605, mean_eps: 0.707815
  4308/50000: episode: 6, dur

In [17]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...


C:\Users\milod\anaconda3\envs\tf_gpu\lib\site-packages\gym\envs\atari\environment.py:255: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(


Episode 1: reward: 500.000, steps: 731
Episode 2: reward: 310.000, steps: 527
Episode 3: reward: 180.000, steps: 598
Episode 4: reward: 240.000, steps: 556
Episode 5: reward: 1160.000, steps: 734
Episode 6: reward: 490.000, steps: 934
Episode 7: reward: 420.000, steps: 752
Episode 8: reward: 400.000, steps: 713
Episode 9: reward: 310.000, steps: 655
Episode 10: reward: 340.000, steps: 639
435.0


In [ ]:
dqn.save_weights('SavedWeights_v2/10k-Fast/dqn_weights.h5f')

In [50]:
del model, dqn

NameError: name 'model' is not defined

In [16]:
dqn.load_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

https://stable-baselines3.readthedocs.io/en/master/common/atari_wrappers.html  
https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf  
https://www.nature.com/articles/nature14236  
### GPU
### PreProces
### Mniejsza sieć

In [17]:
with tf.device('/GPU:0'):
    dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)
dqn.save_weights('SavedWeights_v2/10k-Fast/dqn_weights.h5f')

Training for 50000 steps ...


C:\Users\milod\anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


   565/50000: episode: 1, duration: 11.333s, episode steps: 565, steps per second:  50, episode reward: 130.000, mean reward:  0.230 [ 0.000, 10.000], mean action: 3.940 [0.000, 8.000],  loss: --, mean_q: --, mean_eps: --


C:\Users\milod\anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  1261/50000: episode: 2, duration: 138.460s, episode steps: 696, steps per second:   5, episode reward: 340.000, mean reward:  0.489 [ 0.000, 10.000], mean action: 4.066 [0.000, 8.000],  loss: 2.211012, mean_q: 20.425924, mean_eps: 0.898255
  1856/50000: episode: 3, duration: 307.834s, episode steps: 595, steps per second:   2, episode reward: 220.000, mean reward:  0.370 [ 0.000, 10.000], mean action: 4.047 [0.000, 8.000],  loss: 0.875234, mean_q: 20.873092, mean_eps: 0.859780
  2740/50000: episode: 4, duration: 456.046s, episode steps: 884, steps per second:   2, episode reward: 380.000, mean reward:  0.430 [ 0.000, 10.000], mean action: 3.896 [0.000, 8.000],  loss: 1.151486, mean_q: 20.930360, mean_eps: 0.793225
  3395/50000: episode: 5, duration: 341.502s, episode steps: 655, steps per second:   2, episode reward: 290.000, mean reward:  0.443 [ 0.000, 10.000], mean action: 3.823 [0.000, 8.000],  loss: 0.873133, mean_q: 21.115546, mean_eps: 0.723970
  4109/50000: episode: 6, durati

In [19]:
model.save("Pacman_v2")

INFO:tensorflow:Assets written to: Pacman_v2\assets


In [ ]:
del model, dqn

In [21]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))
env.close()

Testing for 10 episodes ...
Episode 1: reward: 990.000, steps: 796
Episode 2: reward: 160.000, steps: 453
Episode 3: reward: 430.000, steps: 819
Episode 4: reward: 370.000, steps: 622
Episode 5: reward: 880.000, steps: 955
Episode 6: reward: 370.000, steps: 1042
Episode 7: reward: 470.000, steps: 1017
Episode 8: reward: 460.000, steps: 844
Episode 9: reward: 1240.000, steps: 1001
Episode 10: reward: 510.000, steps: 704
588.0
